<a href="https://colab.research.google.com/github/ItaloHnrq33/DWEB2/blob/main/Classification_Minera%C3%A7%C3%A3o_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas-profiling==2.7.1

!pip install pycaret

In [2]:
import pandas as pd
import numpy as np
import pandas_profiling
from sklearn import model_selection
from sklearn.utils import resample
from sklearn.preprocessing import scale
from sklearn.ensemble import *
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score
from pycaret.classification import setup
from pycaret.classification import compare_models

In [3]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00571/hcvdat0.csv"
df = pd.read_csv(url, delimiter=',', index_col=0)
df.head()

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
1,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
2,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
3,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
4,0=Blood Donor,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
5,0=Blood Donor,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7


In [4]:
categories = df.Category.unique()
categories

array(['0=Blood Donor', '0s=suspect Blood Donor', '1=Hepatitis',
       '2=Fibrosis', '3=Cirrhosis'], dtype=object)

In [5]:
df.Category.replace(categories, [0,0,1,1,1], inplace=True)


In [6]:
#Transformando de um dado categórico para binário
df['Sex'] = df['Sex'].map({'m': 0, 'f': 1})
df

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
1,0,32,0,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
2,0,32,0,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
3,0,32,0,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
4,0,32,0,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
5,0,32,0,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,1,62,1,32.0,416.6,5.9,110.3,50.0,5.57,6.30,55.7,650.9,68.5
612,1,64,1,24.0,102.8,2.9,44.4,20.0,1.54,3.02,63.0,35.9,71.3
613,1,64,1,29.0,87.3,3.5,99.0,48.0,1.66,3.63,66.7,64.2,82.0
614,1,46,1,33.0,NaN,39.0,62.0,20.0,3.56,4.20,52.0,50.0,71.0


In [7]:
#chequado se exixte dados nulos
df.isnull().sum()

Category     0
Age          0
Sex          0
ALB          1
ALP         18
ALT          1
AST          0
BIL          0
CHE          0
CHOL        10
CREA         0
GGT          0
PROT         1
dtype: int64

In [8]:
#Deletando os dados que são nulos
df = df.dropna()
df.shape

(589, 13)

In [9]:
#Analisando a média/desvio padrão
df.describe()

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
count,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000
mean,0.095076,47.417657,0.383701,41.624278,68.123090,26.575382,33.772835,11.018166,8.203633,5.391341,81.669100,38.198472,71.890153
std,0.293570,9.931334,0.486700,5.761794,25.921072,20.863120,32.866871,17.406572,2.191073,1.128954,50.696991,54.302407,5.348883
min,0.000000,23.000000,0.000000,14.900000,11.300000,0.900000,10.600000,0.800000,1.420000,1.430000,8.000000,4.500000,44.800000
25%,0.000000,39.000000,0.000000,38.800000,52.500000,16.400000,21.500000,5.200000,6.930000,4.620000,68.000000,15.600000,69.300000
50%,0.000000,47.000000,0.000000,41.900000,66.200000,22.700000,25.700000,7.100000,8.260000,5.310000,77.000000,22.800000,72.100000
75%,0.000000,54.000000,1.000000,45.100000,79.900000,31.900000,31.700000,11.000000,9.570000,6.080000,89.000000,37.600000,75.200000
max,1.000000,77.000000,1.000000,82.200000,416.600000,325.300000,324.000000,209.000000,16.410000,9.670000,1079.100000,650.900000,86.500000


In [10]:
#
def limitar_anomalias(data, anomalia_cols):
  for col in anomalia_cols:
    q25, q75 = np.percentile(data[col], 25), np.percentile(data[col], 75)

    iiq = q75 - q25

    print(f'Percentis coluna {col}: 25% = {q25:.3f}, 75% = {q75:.3f}, IIQ = {iiq:.3f}')

    corte = iiq * 1.5

    inferior, superior = q25 - corte, q75 + corte

    anomalias = [x for x in data[col] if x < inferior or x > superior]

    print(f'Outliers Identificados na coluna {col}: {len(anomalias)}')
    print()

    data[col] = np.where(data[col] > superior, superior, 
                         np.where(data[col] < inferior, inferior, 
                                  data[col]))
  
  return data

#função para normalização
def normalizar(data, normalizar_cols):
  for col in normalizar_cols:
    data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())

  return data

In [ ]:
#setando os atributos para serem 
anomalia_cols = ['ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']
df = limitar_anomalias(df.copy(), anomalia_cols)

df.describe()

In [ ]:
#setando os atributos a serem normalizados
normalizar_cols = ['ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']

df = normalizar(df.copy(), anomalia_cols)

df.describe()

In [13]:
#checando se a dados duplicados
duplicados = df.duplicated()
print(duplicados.any())

False


In [14]:
#realizando o balanceamento das classes utilizando o oversample
mascara = df.Category == 0
df_0 = df[mascara]
df_1 = df[~mascara]
df_oversample = resample(df_1, n_samples=len(df_0), random_state=13)
df2 = pd.concat([df_0, df_oversample])
df2.Category.value_counts()

1    533
0    533
Name: Category, dtype: int64

In [15]:
#checando novamente se há dados duplicados após o balanceamento
duplicados = df2.duplicated()
print(duplicados.any())

True


In [16]:
#inicializando o Pycaret
clf = setup(data=df, target='Category', silent=True, verbose=False, log_experiment=True, session_id=13)
#pegando os 5 melhores modelos
best = compare_models(n_select=5)
for model in best:
  print(model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9902,1.0000,0.900,1.0000,0.9381,0.9332,0.9398,0.121
lda,Linear Discriminant Analysis,0.9878,0.9885,0.900,0.9800,0.9270,0.9207,0.9280,0.018
ridge,Ridge Classifier,0.9854,0.0000,0.850,1.0000,0.9048,0.8975,0.9087,0.016
qda,Quadratic Discriminant Analysis,0.9854,0.9736,0.875,0.9800,0.9079,0.9006,0.9114,0.018
rf,Random Forest Classifier,0.9830,0.9980,0.850,0.9800,0.8984,0.8896,0.8990,0.512
lightgbm,Light Gradient Boosting Machine,0.9830,0.9993,0.850,0.9800,0.8984,0.8896,0.8990,0.093
gbc,Gradient Boosting Classifier,0.9806,0.9967,0.850,0.9600,0.8825,0.8727,0.8852,0.139
dt,Decision Tree Classifier,0.9757,0.8973,0.800,0.9600,0.8492,0.8370,0.8540,0.017
lr,Logistic Regression,0.9733,0.9892,0.725,1.0000,0.8238,0.8107,0.8319,0.357
et,Extra Trees Classifier,0.9709,0.9980,0.700,1.0000,0.8095,0.7951,0.8174,0.473


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=13)
LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)
RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=13, solver='auto',
                tol=0.001)
QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
                              store_covariance=False, tol=0.0001)
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, 

In [17]:
#inicialiando os modelos 1 e 2 
model1 = [AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=13), 'AdaBoost']
model2 = [RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=13, verbose=0,
                       warm_start=False),'Random Forest']
#colocanod na lista para usar na validação cruzada
models = [model1, model2]

In [ ]:
y = df['Category'].values
X = df.drop(columns='Category').values

In [ ]:
#realizando o oversampling somente nos dados de treino
def overSampling(X_train, y_train):
  mascara = y_train == 0
  y_0 = y_train[mascara]
  y_1 = y_train[~mascara]
  X_0 = X_train[mascara]
  X_1 = X_train[~mascara]
  y_oversample = resample(y_1, n_samples=len(y_0))
  X_oversample = resample(X_1, n_samples=len(X_0))
  y_train = np.concatenate((y_0, y_oversample))
  X_train = np.concatenate((X_0, X_oversample))
  # print(f'Class 0 / 1 quant: {y_train[y_train == 0].size} / {y_train[y_train == 1].size}')

  return X_train, y_train


In [ ]:
#executando a validação cruzada
skf = StratifiedKFold(n_splits=10)

for mod in models:
    acc = []
    f1 = []
    precision = []

    for train_indexs, test_indexs in skf.split(X, y):
        X_train, y_train = overSampling(X[train_indexs,:], y[train_indexs])
        X_test, y_test = X[test_indexs,:], y[test_indexs]

        model = mod[0]
        model.fit(X_train, y_train)
        y_predicted = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_predicted)
        f1_scr = f1_score(y_test, y_predicted)
        prec_score = precision_score(y_test, y_predicted)
        acc.append(accuracy)
        f1.append(f1_scr)
        precision.append(prec_score)

    print(f"{mod[1]}, Accuracy:\t{np.mean(acc):.2f} +/- {np.std(acc):.4f}")
    print(f"{mod[1]}, F1 Score:\t{np.mean(f1):.2f} +/- {np.std(f1):.4f}")
    print(f"{mod[1]}, Precision:\t{np.mean(precision):.2f} +/- {np.std(precision):.4f}")
    print("-"*60)
